# 1. Import Packages

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.max_rows', None)  # 显示所有行
import numpy as np
#import jieba
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# 2. Label Match cross multi sheets

In [ ]:
def add_lable():
    '''
    输入：深圳市初始表
    输出: 为每一个样本匹配MMSElable
    '''
    #table1 = pd.read_excel("副本AD8TJ-1205-已匹配-脱敏-加密.xlsx", sheet_name=0)
    table1=pd.read_csv("en20k1.csv")
    print(table1.columns)
    table2 = pd.read_excel("副本AD8TJ-1205-已匹配-脱敏-加密.xlsx", sheet_name=1, usecols=['识别编码', 'MMSE'])
    #table2.rename(columns={'识别编码': table1.columns[0]})
    print(table2.columns)
    table3 = pd.read_excel("副本AD8TJ-1205-已匹配-脱敏-加密.xlsx", sheet_name=2, usecols=['识别编码', 'MMSE最后得分'])
    #table3.rename(columns={'识别编码':table1.columns[1]})
    print(table3.columns)
    table_final= pd.merge(table1, table2, left_on='识别编码-MMSE', right_on='识别编码',how='left')
    table_final = pd.merge(table_final, table3,  left_on='识别编码-失能失智', right_on='识别编码', how='left')
    table_final['MMSESCORE'] = table_final['MMSE'].fillna(table_final['MMSE最后得分'])
    table_final=table_final.drop(columns=['MMSE', 'MMSE最后得分','识别编码-MMSE',"识别编码-失能失智",'识别编码_x', '识别编码_y'])
    print(table_final.shape[0]-table_final["MMSESCORE"].isnull().sum())
    table_final['ID'] = table_final.index
    table_final.to_csv("0.label_added.csv",encoding="utf-8",index_label=False)
add_lable()

# 3. Special Columns Process 

In [ ]:

''' 

* 以下项目来源于体检数据条目明细-关于印发..
* 阅读excel文件，初步认识数据表与理解缺失原因
    * 吸烟情况
    * 饮酒状况只有饮酒频率可以有
    * 牙齿情况————>不健康牙齿数量
    * 尿酮体等四项指标 ——-->标签转换
    * 文本型特征的抽取---->指定关键词进行转换
    * 视力转换 
    * 总胆红素的填充
* 直接修改原有的df!
'''

"""
pandas的数值类型
*  连续型变量（Continuous Variables）：这些变量表示的是可以采用连续值的数据，例如身高、体重、温度等
* 分类变量（Categorical Variables）：分类变量可以是独热编码（One-Hot Encoding）或者序列编码（Ordinal Encoding）。
*  文本字符变量（Text/String Variables）
* 时间型变量和地理空间型变量

"""

def special_Char(df):
    #先计算年龄列
    df['出生日期'] = pd.to_datetime(df['出生日期'])
    #print(unlabeled.columns)
    if '年龄' in df.columns:
        pass
    else:
        
        df['体检日期'] = df['体检日期'].str.replace('月', '').str.replace(' ', '')
        df['体检日期'] = pd.to_datetime(df['体检日期'], format="%d-%m-%y")
        df['年龄'] = (df['体检日期'] - df['出生日期']).dt.days // 365
        df=df.drop(columns=['出生日期','体检日期'],axis=1)
    df.head()


    #特殊列--吸烟状况
    df_a=df[['吸烟状况','戒烟年龄','年龄','日吸烟量','开始吸烟年龄']]
    #print(df_a)
    new_df=pd.dfFrame(0,index=pd.dfFrame(df_a).index, columns=['吸烟总支数', '吸烟年龄', '戒烟行为'])
    # 空缺值都是因为从不吸烟导致日吸烟量 开始吸烟年龄和戒烟年龄为空，所以全部为0
    new_df.loc[pd.dfFrame(df_a)['吸烟状况']=='从不吸烟'] = 0
    new_df.loc[pd.dfFrame(df_a)['吸烟状况']=='已戒烟','戒烟行为'] = 1
    new_df.loc[pd.dfFrame(df_a)['吸烟状况']=='已戒烟','吸烟年龄'] = df_a['戒烟年龄'] -df_a['开始吸烟年龄'] 
    new_df.loc[pd.DataFrame(df_a)['吸烟状况']=='已戒烟','吸烟总支数'] = new_df['吸烟年龄']* 365 * df_a['日吸烟量']
    new_df.loc[pd.DataFrame(df_a)['吸烟状况']=='吸烟','戒烟行为']=0
    new_df.loc[pd.DataFrame(df_a)['吸烟状况']=='吸烟','吸烟年龄']= df['年龄'] - df['开始吸烟年龄']
    new_df.loc[pd.DataFrame(df_a)['吸烟状况']=='吸烟','吸烟总支数']= new_df['吸烟年龄'] * 365 * df_a['日吸烟量']

    insert_index = df.columns.get_loc('吸烟状况')
    df=df.drop(['吸烟状况','戒烟年龄','日吸烟量','开始吸烟年龄'],axis=1)
    for col in new_df.columns:
        df.insert(insert_index, col, new_df[col])


    #特殊列--饮酒状况  大多数缺失比率>0.8 只有饮酒频率是0.04缺失比率
    missing_ratio = df[["饮酒频率","日饮酒量","是否戒酒","戒酒年龄","开始饮酒年龄","近一年内是否醉酒","饮酒种类"]].isnull().mean()
    #selected_columns = missing_ratio[missing_ratio < 0.8].index.tolist()
    #print("缺失比率小于0.8的列：", selected_columns)
    df=df.drop(["日饮酒量","是否戒酒","戒酒年龄","开始饮酒年龄","近一年内是否醉酒","饮酒种类"],axis=1)



    # 特殊列--牙齿情况 
    '''有删除样本的需要先进行操作'''
    #print(df.shape[0])  #未删减错误前样本量
    df1=df[['缺齿左上', '缺齿左下', '缺齿右上', '缺齿右下', 
            '龋齿左上','龋齿左下', '龋齿右上', '龋齿右下', 
            '义齿左上', '义齿左下', '义齿右上', '义齿右下']]
    df1 = df1[~df1.apply(lambda x: x.str.contains(r'\d+月\d+日|(\d+-\d+)')).any(axis=1)]
    #print(df1.iloc[58:62,:])   # 删除输错样本 错误特征：含月或者日
    df1 = df1.applymap(lambda x: len(str(x).replace("-", "")) if pd.notnull(x) and isinstance(x, str) else len(str(x)) if pd.notnull(x) else 0)#牙齿转换成缺损数量 若全部为0则表示是好的牙齿
    #print(df1.shape[0])  #删减错误后的样本量
    df[['缺齿左上', '缺齿左下', '缺齿右上', '缺齿右下', '龋齿左上',
        '龋齿左下', '龋齿右上', '龋齿右下', '义齿左上', '义齿左下', '义齿右上', '义齿右下']]=df1.fillna(0)



    # 特殊列——左右眼视力
    df_eye=df[['左眼视力', '右眼视力','左眼矫正视力','右眼矫正视力']]
    df_eye['左眼矫正历史'] = df_eye.apply(lambda row: 1 if pd.isna(row['左眼视力']) else 0, axis=1)
    df_eye['右眼矫正历史'] = df_eye.apply(lambda row: 1 if pd.isna(row['右眼视力']) else 0, axis=1)
    df_eye['左眼视力'] = df_eye.apply(lambda row: row['左眼矫正视力'] if pd.isna(row['左眼视力']) else row['左眼视力'], axis=1)
    df_eye['右眼视力'] = df_eye.apply(lambda row: row['右眼矫正视力'] if pd.isna(row['右眼视力']) else row['右眼视力'], axis=1)
    df_eye.drop(columns=['左眼矫正视力', '右眼矫正视力'], inplace=True)
    df_eye.iloc[:5,:]
    df[['左眼视力', '右眼视力','左眼矫正视力','右眼矫正视力']]=df_eye

    #特殊列--- 总胆红素
    #总胆红素=结合胆红素+非结合胆红素  可以做部分缺失值填充   后两者还是14w左右的空缺值因此只填充总胆红素
    df_bilirubin=df[['总胆红素','结合胆红素','非结合胆红素']]
    missing_indexes = df_bilirubin[df_bilirubin['总胆红素'].isnull()].index
    for index in missing_indexes:
        df_bilirubin.at[index, '总胆红素'] = df_bilirubin.at[index, '结合胆红素'] + df_bilirubin.at[index, '非结合胆红素']
    df['总胆红素'] = df_bilirubin['总胆红素']


    #  文本： 特殊列--心电图异常  眼底   异常点为1，全是0则表示正常
    data=[str(x) for x in df['心电图异常情况'].to_list()]
    # 初始化特征字典
    features = {
        "心电图心律类型": [],
        "心电图心率": [],
        "心电图ST-T改变": [],
        "心电图心电轴": [],
        "心电图T波改变": [],
        "心电图房室传导阻滞": [],
        "心电图左心室电压": [],
        "心电图心房颤动": []
    }
    for item in data:
        # 心律类型
        if "窦性心律" in item:
            features["心电图心律类型"].append("0")
        elif "异性心律" or "异位心律" in item:
            features["心电图心律类型"].append("2")
        else:
            features["心电图心律类型"].append("1")

        # 心率
        if "心动过缓" in item:
            features["心电图心率"].append("0")
        elif "频发" in item:
            features["心电图心率"].append("2")
        else:
            features["心电图心率"].append("1")

        # 其他特征提取示例，可根据具体情况添加
        features["心电图ST-T改变"].append("1" if "ST-T改变" in item else 0)
        features["心电图心电轴"].append("1" if "左偏" in item else "0")
        features["心电图T波改变"].append("1" if "T波改变" in item else "0")
        features["心电图房室传导阻滞"].append("1" if "传导阻滞" in item else "0")
        features["心电图左心室电压"].append("1" if "高电压" in item else "0")
        features["心电图心房颤动"].append("1" if "心房颤动" in item else "0")
    df=df.drop(columns=["心电图"],axis=1)

    # 构建特征表格
    df_xindian = pd.dfFrame(features)
    # 打印特征表格
    print(df_xindian.head())
    insert_index = df.columns.get_loc('心电图异常情况')
    df=df.drop("心电图异常情况",axis=1)
    for col in df_xindian.columns:
        df.insert(insert_index, col, df_xindian[col])

    # 文本 特殊列 ---腹部B超原因 异常点为1，全是0则表示正常
    data = [str(x) for x in df["腹部B超异常原因"].tolist()]
    df_abnormal_belly = pd.DataFrame()
    keywords = ['脂肪肝', '囊肿', '胆囊壁隆起', '息肉', '胆囊泥沙样结石', '内沉积物', '肝内钙化灶', '血管瘤', '胆总管增宽', '胆结石', '胆总管扩张', '胆囊切除术后', '胆囊多发性息肉', '胆附壁结石', '胆充满型结石', '前列腺钙化斑形成']
    for keyword in keywords:
        df_abnormal_belly["腹部"+keyword] = pd.Series([1 if keyword in x else 0 for x in data])
    df_abnormal_belly.head()
    insert_index = df.columns.get_loc('腹部B超异常原因')
    df=df.drop("腹部B超异常原因",axis=1)
    for col in df_abnormal_belly.columns:
        df.insert(insert_index, col, df_abnormal_belly[col])

    # 文本 特殊列--双肾B超原因  异常点为1，全是0则表示正常
    samples = [str(x) for x in df["双肾B超异常原因"].tolist()]
    keywords = ['错构瘤', '高回声', '囊肿', '结石', '肾柱']
    df_kidneys = pd.DataFrame(columns=['双肾'+x for x in keywords])
    for keyword in keywords:
        df_kidneys['双肾'+keyword] = pd.Series([1 if keyword in x else 0 for x in data])
    print(df_kidneys.head())
    insert_index = df.columns.get_loc('双肾B超异常原因')
    df=df.drop("双肾B超异常原因",axis=1)
    for col in df_kidneys.columns:
        df.insert(insert_index, col, df_kidneys[col])



    # 文本 特殊列---胸部X线片异常原因
    data = [str(x) for x in df["异常原因"].tolist()]
    df_abnormal_breast = pd.DataFrame()
    keywords = ['主动脉硬化', '心影增大', '肺纹理增多', '肺结节', '肺炎症', '胸主动脉粥样硬化', '气管受压', '冠脉支架植入术后', '老年性肺间质改变', '胸椎侧弯', '脊柱侧弯畸形']
    for keyword in keywords:
        df_abnormal_breast["胸"+keyword] = pd.Series([1 if keyword in x else 0 for x in data])
    insert_index = df.columns.get_loc('异常原因')
    df=df.drop("异常原因",axis=1)
    for col in df_abnormal_breast.columns:
        df.insert(insert_index, col, df_abnormal_breast[col])

    # 符号特殊列---有空缺列
    #尿蛋白
    def map_protein_value(label):
        """
        尿蛋白的医学知识
        "-"：0（阴性）
        "±"：0.15（弱阳性）
        "1+"：0.3
        "2+"：1
        "3+"：3
        "4+"：5
        """
        mapping = {'-': 0, '+': 0.3, '±': 0.15, '+-': 0.15, '阴性': 0, '++': 1, '—': 0, '+3': 3, None: None, '+++': 3, '+1': 0.3, '+2': 1, '3+': 3, '+—': 0.15, '+|-': 0.15, '--': 0, '++++': 5, '——': 0, '弱阳性': 0.15, '2+': 1, '阳性': 0.15, '（-）': 0, '1+': 0.3, '0': 0, '+——': 0.15, '15': 0, '+ -': 0.15, '+ -  0.15': 0.15, '70': 5, '（+）': 0.3, '=': None, '弱阳': 0.15, '微量': 0.15, '1.71': 5, '（—）': 0, '－': 0, '+/-': 0.15, '（+—）': 0.15, '(—）': 0.15, '（+-）': 0.15, '（++）': 1, '(+-)': 0.15, '＋-': 0.15, '阴性阴性': 0, '（—)': 0, '正常': 0, '（+++）': 5, '95.5': None, '(-)': 0, '0.3': 0.3, '（++)': 1, '4+': 5, '(++)': 1, '0.15': 0.15, '(—)': 0, '11.06': 5, '一': 0, '2加': 1, '放弃': None, '加': 0.3, '＋＋': 1, '---': None, '＋－': 0.15, '-+': 0.15, '+一': 0.15, '+/—': 0.15, '未做': None, '士': 0.15, '＋': 0.3, '阴性-': 0, '阴性（-）': 0, '＋＋＋': 3, '阴-': 0, '|+': 0.3, '-0': 0, '（---）': None, '（--）': None, '1': 0.3, '-0.15': 0.15, '0.15g/L': 0.15, '（---)': None, '2-': 1, '1++': 0.3, '9': 5, '+1-': 0.3, '1.0++': 0.3, '/': None, '-1': 0, '-`': 0, '----': None, '（--)': 0, '0.15+': 0.15, '25（+）': 5, '25+': 5, '6.65': 5, '1（++）': 1, '0.15（+）': 0.15, '（+2）': 1, '+1  0.3g/L': 0.3, '+- 0.15': 0.15, '0.3（+）': 0.3, '2＋': 1, '！+': 0.3, '（+-)': 0.15, '+－': 0.15, '＋/-': 0.15, '0.3+': 0.3, '（+3）': 3, '-l': 0, '75++': 1, '500+++': 5, '-·': 0, '*': None, '少许': 0.15, '+ —': 0.15, '35.3': None, '+--': 0.15, '15mg/dl': 5, '（+ -）': 0.15, '（-)': 0, '(--)': 0, '+ 一': 0.15, '（+1）': 0.3, '（——）': 0, '++-': 1, '100mg/dl': 5, '―': 0, '阴性（—）': 0, '+ + +': 3, '2+·-': 1, '无法留尿': None, '阴': 0, '91.1': 5, '—+': 0.15, '1-': 0, '2+·': 1, '2': 1, '阴性(-)': 0, '（2+）': 1, '+4': 5}
        return mapping.get(label)
    urine_protein_data = df["尿蛋白"].to_list()
    df["尿蛋白"] = pd.DataFrame([map_protein_value(value) for value in urine_protein_data])

    #print(converted_data[:50])
    #pd.DataFrame(converted_data).iloc[:,0].unique()

    #尿糖
    def transform_urine_sugar(value):
        """尿糖的医学知识
    - 颜色不变 阴性:尿内无糖
    ± 不变色,冷却后有少许绿色沉淀 微量:少于0.25g/dl  250mg
    + 略有颜色改变 少量:0.25-0.50g/dl 500mg
    ++ 绿黄色 中量:约0.5-1g/dl  1000mg
    +++ 土黄色 大量:1-2g/dl  2000mg
    ++++ 棕红色 极大量:>2g/dl　　   2000mg
    """
        if value in ['阴虚', '-阴性', '（-））', '(-）', '阴性(-)', '性（-）', '阴', '阴性（—）', '―', '—·', '（-)', '·-', '-', '阴性', '—', '--', '（-）', '（—）', '－', '(—）', '（—)', '正常', '—）', '(—)', '一', '阴性-', '阴-', '阴性（-）', '-0', '0']:
            return 0
        elif value in ['弱阳性。', '1＋', '（+1）', '（1+）', '阳阴性', '+ —', '—+', '8.5', '(+-)', '55mmol', '55mm0l/L', '28', '55mmol/L', '100mg/dl', '50ml/dl', '(+)', '＋1', '＋-', '+', '+', '+1', '+-', '+—', '弱阳性', '阳性', '微量', '=', '（+—）', '（+）', '＋－', '加', '+/-', '-+', '1=', '+一', '28+', '28mmol/L', '2+-', '+-2.8', '2.8+', '5.6+', '-+-', '2.8', '2.8（+）', '1']:
            return 0.5
        elif value in ['2＋', '（2+）', '2*', '++', '+2', '2+', '(++)', '（++）', '2加', '＋＋', '2', '（+2）', '14（++）', '28（++）']:
            return 1
        elif value in ['阳性+++', '（3+）', '（+3）', '+ + +', '+++', '+3', '3+', '1.030', '＋＋＋', '3加', '+＋＋', '28（+++）']:
            return 2
        elif value in ['(+4)', '3.84', '（4+）', '（+4）', '++++', '+4', '4+', '（++++）', '+4 55', '4加', '＋＋＋＋', '4＋', '14++', '+3.28', '28+++', '4.80', '4', '8672.7']:
            return 5
        else:
            return None  #[  '？（-）','BAS', '无法留尿','（——）','（）','(--)','放弃','未做','（---）','-——', '（--）','*','（---)', '/','（--)','拒查', '未检'，'---',]"""
    # 应用转换函数到数据列
    df['尿糖'] = df['尿糖'].apply(transform_urine_sugar)


    #尿酮体
    urineacetone_data = [
        ['阴性', 0, 0],
        ['微量', 5, 0.5],
        ['+', 151.5, 1.5],
        ['+ +',  40, 3.9],
        ['+ + +', 80, 7.8],
        ['+ + + +',  160, 15.6]
    ]
    criteria_urineacetone  = pd.DataFrame(urineacetone_data, columns=['尿糖',  'mg/dL', 'mmol/L'])
    #print("尿酮体转换标准：",criteria_urineacetone)
    #df["尿酮体"].unique()

    def transform_urine_acetone(value):
        if value in [ '阴性(-)', '_', '(-）', '（-））', '-阴性',  '性（-）','---', '-/','阴','阴性（—）', '（-)', '(--)', '阴隆', '（——）', '―', '-—','-*','（--)','（---)','阴怀','0', '-0','阴-','-', '阴性', '—',  '（-）', '(-)', '（—）', '－','(—）','（—)','正常', '(—)','一','阴性-', '--','——', '（---）', '（--）',]:
            return 0
        elif value in [ '+ —', '—+','15md/dl', '+－','-+-','+-0.5', '-+2', '-+','弱阳性','微量','弱阳', '阴性（-）','-·', '+/-', '+-', '+—','±', '+ -', '（+—）', '（+-）', '+—）','＋－', '+/—','士',]:
            return 0.5
        elif value in [ '阳性', '阳阴性','1.0（+）', '（+1）','1', '+','+1', '1+',  '（+）',  '加','1.0+', ]:
            return 1.5
        elif value in ['++','2+','+2', '2.5+','10（++）', '＋2',]:
            return 3.9
        elif value in ['+++','+3', '3+',]:
            return 7.8
        elif value in ['++++', '（+4）', ]:
            return 15.6
        else:
            return None #nan, '=','同学', '*','放弃', '未做','\u3000',   '/', '无法留尿','拒查', '未检'
    df["尿酮体"]=df["尿酮体"].apply(transform_urine_acetone)

    #尿潜血
    def transform_urine_occult_blood(value):
        if value in ['-', '—', '（-）', '——', '--', '（---）', '（--）', '（-））', '-阴性', '阴性-', '-/']:
            return 0
        elif value in ['+', '+—', '+—）', '+ —', '+- 10', '阴性（—）', '+/—', '+－', '15md/dl', '-+-', '+—', '+-', '+-', '+—', '阴性（-）', '+ -', '阴性(-)', '阴', '-+', '-+', '-+', '-+1', '+—）', '士', '微量', '弱阳', '弱阳性']:
            return 1
        elif value in ['1', '1+', '1=', '加', '+', '1+', '+1', '＋', '+1.25', '+1、', '1+。', '（+）', '阳性（+）', '1.0+', '阳性', '+1', '+－', '+1、', '+一', '＋1', '加']:
            return 1
        elif value in ['++', '2+', '2＋', '＋2', '++=', '（++）', '2＋', '2加', '2+-']:
            return 2
        elif value in ['+++', '3+', '+3', '3++', '（+3）', '3＋', '3加', '3+++']:
            return 3
        elif value in ['++++', '大于3+', '3+++', '300+++', '+++＞200', '+24.3', '50++', '（+++）']:
            return 4
        else:
            return None
    df["尿潜血"]=df["尿潜血"].apply(transform_urine_occult_blood)
    #df["尿潜血"].value_counts()

    # 饮食习惯
    df_food= pd.DataFrame(df["饮食习惯"])
    all_habits = ['素食为主', '荤食为主', '嗜盐', '嗜油', '嗜糖']
    for habit in all_habits:
        df_food[habit] = 0
    for index, row in df.iterrows():
        habits = str(row['饮食习惯']).split(',')
        for habit in habits:
            habit = habit.strip()  # 去除习惯中的空格
            if habit in all_habits:
                df_food.at[index, habit] = 1
    insert_index = df.columns.get_loc('饮食习惯')
    df=df.drop("饮食习惯",axis=1)
    for col in df_food.columns:
        df.insert(insert_index, col, df_food[col])
        
    df_food.drop(columns=['饮食习惯'], inplace=True)
    #print(df.columns)

    # 皮肤
    # 已定义的body_parts和skin_diseases列表
    body_parts = [
        "头部", "脸部", "颈部", "前胸", "背部", "腹部", "臀部",
        "上肢", "下肢", "手臂", "前臂", "手掌", "手背", "手指",
        "大腿", "小腿", "脚踝", "脚背", "脚掌", "脚趾",
        "左膝", "右膝", "左肩", "右肩", "左臂", "右臂",
        "左手", "右手", "左脚", "右脚",
        "眼周", "嘴唇", "耳朵", "鼻子", "眉毛", "眼睛", "舌头",
        "腋下", "私处", "乳房", "腰部", "肩膀", "肘部", "膝盖",
        "腕部", "颈后", "脚跟", "脚尖", "脚底", "脚趾甲", "手指甲"
    ]
    skin_diseases = [
        "色素沉着", "皮下毛细血管曲张", "白癜风", "皮脂腺囊肿", "湿疹",
        "手术疤痕", "皮肤干燥", "斑块", "肿物", "蜘蛛痣", "坏死", "发绀",
        "色素脱失斑", "缺如", "指凹性水肿", "瘀青肿胀","湿疹", "银屑病", 
        "痤疮", "皮肤癌", "白癜风",
        "荨麻疹", "带状疱疹", "脂溢性皮炎", "足癣",
        "手足口病", "疱疹", "黄褐斑", "扁平疣", "疣",
        "鱼鳞病", "红斑狼疮", "玫瑰痤疮", "毛囊炎",
        "脱发", "斑秃", "汗斑", "色素沉着", "皮肤干燥",
        "冻疮", "日光性皮炎", "皮肤过敏", "接触性皮炎",
        "蜂窝组织炎", "褥疮", "瘙痒症", "肉芽肿",
        "系统性硬化症", "多形红斑", "皮肤结核", "皮肤真菌感染"
    ]

    # 定义提取位置和症状的函数
    def extract_location_and_symptom(description, body_parts, skin_diseases):
        location = "None"
        symptom = "None"
        if not isinstance(description, str):
            return location, symptom  # 如果不是字符串，直接返回默认值
        for part in body_parts:
            if part in description:
                location = part
                break
        for disease in skin_diseases:
            if disease in description:
                symptom = disease
                break
        return location, symptom

    # 应用函数提取信息
    df["皮肤病位置"], df["皮肤病症状"] = zip(*df["皮肤"].apply(lambda x: extract_location_and_symptom(x, body_parts, skin_diseases)))

    # 获取"皮肤"列的位置
    skin_col_index = df.columns.get_loc("皮肤")

    # 删除原始的"皮肤"列
    df.drop("皮肤", axis=1, inplace=True)

    # 将新列插入到原始"皮肤"列的位置
    df.insert(skin_col_index, "皮肤病症状", df.pop("皮肤病症状"))
    df.insert(skin_col_index, "皮肤病位置", df.pop("皮肤病位置"))



    #眼底
    eye_parts = ["左眼", "右眼", "双眼"]
    eye_diseases_unique = [
        "黄斑变性", "乳纹状眼底", "硬性渗出", "地图样萎缩", "豹纹眼底", "青光眼", "黄斑病变",
        "病理性近视", "糖尿病视网膜病变", "视盘水肿", "斑块萎缩", "视网膜内积液", "老年性白内障",
        "玻璃膜疣", "视网膜脱落", "视网膜积液", "视网膜上皮脱离", "玻璃体牵拉", "视网膜萎缩",
        "玻璃膜瘢痕", "棉絮斑", "视网膜动脉硬化", "视网膜玻璃膜疣", "视网膜萎缩灶", "视网膜病变",
        "视网膜静脉阻塞", "视神经萎缩", "瞳孔闭锁", "玻璃体后脱离", "干眼症", "结膜炎", "散光",
        "近视", "远视", "弱视", "角膜炎", "眼睑炎", "高血压视网膜病变", "视神经炎", "眼球震颤",
        "屈光不正", "眼底出血", "眼疲劳", "色盲"
    ]

    # 初始化新列
    df["眼底异常位置"] = ""
    df["眼底异常症状"] = ""

    # 提取信息
    for index, row in df.iterrows():
        position_found = False
        eye_base_info = row["眼底"]
        # 检查eye_base_info是否为字符串
        if isinstance(eye_base_info, str):
            for part in eye_parts:
                if part in eye_base_info:
                    df.at[index, "眼底异常位置"] = part
                    position_found = True
                    break
        else:
            # 如果eye_base_info不是字符串（例如NaN），则标记为"未明确"
            df.at[index, "眼底异常位置"] = "无"
        
        if not position_found:
            df.at[index, "眼底异常位置"] = "无"
        
        # 同样的检查逻辑应用于症状提取
        if isinstance(eye_base_info, str):
            symptoms = [disease for disease in eye_diseases_unique if disease in eye_base_info]
            df.at[index, "眼底异常症状"] = ", ".join(symptoms) if symptoms else "无"
        else:
            df.at[index, "眼底异常症状"] = "无"

    # 获取原“眼底”列的位置
    eye_base_col_index = df.columns.get_loc("眼底")

    # 删除原“眼底”列
    df.drop(columns=["眼底"], inplace=True)
    # 将新列插入到原“眼底”列的位置
    df.insert(eye_base_col_index, "眼底异常位置", df.pop("眼底异常位置"))
    df.insert(eye_base_col_index + 1, "眼底异常症状", df.pop("眼底异常症状"))


    #巩膜
    print(df["巩膜"].unique())
    # 巩膜疾病字典
    sclera_diseases_dict = {
        "正常": ["正常"],
        "充血": ["充血", "双眼结膜充血", "双侧结膜充血", "结膜轻度充血", "左眼巩膜充血"],
        "胬肉": ["翼状驽肉？", "翼壮状胬肉", "双眼轻度翼状胬肉", "结膜：翼状胬肉（双）", "左眼胬肉", "右眼翼状胬肉", "双眼翼状胬肉", "左侧胬肉", "胬肉", "右胬肉", "双侧胬肉", "双侧翼状胬肉", "左侧翼状胬肉"],
        "白内障": ["白内障", "左眼白内障", "双眼白内障", "右眼白内障"],
        "结膜炎": ["慢性结膜炎", "结膜炎"],
        "黄染": ["黄染"],
        "浑浊": ["浑浊", "混浊", "滤泡", "右眼巩膜浑浊"],
        "赘生物": ["左眼肉瘤", "右眼赘生物", "巩膜赘生物"]
    }

    # 巩膜位置列表
    sclera_diseases_loc = ["左眼", "右眼", "双眼", "无"]

    # 初始化新列
    df["巩膜位置"] = ""
    df["巩膜症状"] = ""

    # 提取信息
    for index, row in df.iterrows():
        sclera_info = row["巩膜"]
        position_found = False
        symptoms_found = False
        
        if isinstance(sclera_info, str):  # 检查值是否为字符串类型
            # 检查位置
            for loc in sclera_diseases_loc:
                if loc in sclera_info:
                    df.at[index, "巩膜位置"] = loc
                    position_found = True
                    break
            if not position_found:
                df.at[index, "巩膜位置"] = "无"
            
            # 检查症状
            for category, diseases in sclera_diseases_dict.items():
                for disease in diseases:
                    if disease in sclera_info:
                        df.at[index, "巩膜症状"] = category
                        symptoms_found = True
                        break
                if symptoms_found:
                    break
            if not symptoms_found:
                df.at[index, "巩膜症状"] = "无"
        else:
            df.at[index, "巩膜位置"] = "无"
            df.at[index, "巩膜症状"] = "无"

    # 删除原“巩膜”列
    if "巩膜" in df.columns:
        df.drop(columns=["巩膜"], inplace=True)
    else:
        print("未找到列 '巩膜'")

    # 获取原“巩膜”列的位置
    if "巩膜位置" in df.columns:
        sclera_col_index = df.columns.get_loc("巩膜位置")
    else:
        print("未找到列 '巩膜位置'")

    # 将新列插入到原“巩膜”列的位置
    if "巩膜位置" in df.columns and "巩膜症状" in df.columns:
        df.insert(sclera_col_index, "巩膜位置", df.pop("巩膜位置"))
        df.insert(sclera_col_index + 1, "巩膜症状", df.pop("巩膜症状"))
    else:
        print("未找到列 '巩膜位置' 或 '巩膜症状'")


    #df["呼吸音"].unique()
    def extract_respiratory_info(description):
        position ="无"
        symptom = "无"
        # 定义位置和症状字典
        position_dict = {'右肺': '右肺', '左肺': '左肺', '双肺': '双肺', '上肺': '上肺', '下肺': '下肺', '背部': '背部', '支气管': '支气管'}
        symptom_dict = {'增粗': ['增粗', '粗', '稍粗'], '低': ['减弱', '低', '右肺呼吸音低', '双肺呼吸音低', '左上肺呼吸音低', '双侧呼吸音低', '呼吸音低'], '哮鸣音': ['哮鸣音'], '呼气相延长': ['呼气相延长', '呼气相延长，偶可闻及痰鸣音'], '干啰音': ['干啰音', '双肺干啰音', '左上肺背部可闻及干鸣音', '左侧支气管呼吸音', '左下肺背部可闻及干鸣音', '双肺干啰音，左侧明显'], '湿罗音': ['偶有湿罗音', '湿罗音', '右下肺可闻及湿罗音', '左下肺湿啰音'], '喘鸣音': ['喘鸣音']}
        description = str(description)
        # 遍历位置和症状字典，提取描述中的位置和症状信息
        for key, value in position_dict.items():
            if value in description:
                position = key
                break
        for key, value in symptom_dict.items():
            if any(symptom_word in description for symptom_word in value):
                symptom = key
                break
        return position, symptom

    # 应用函数并将结果拆分为两列
    df['呼吸音位置'], df['呼吸音症状'] = zip(*df['呼吸音'].apply(extract_respiratory_info))



    #df["淋巴结"].unique()
    valid_locations = {
        '颈部': ['颈下', '颏下', '颈动脉附近', '颈动脉', '颈', '颈部', '锁骨'],
        '腹部': ['腹股沟区', '腹部'],
        '胸部': ['腹股沟区', '腹部', '胸部'],
        '腋窝': ['腋窝', '上肢前臂内侧'],
        '颌下': ['下颌', '下颌下', '下颌面', '下颌角'],
        "正常": ["正常"]
    }

    def process_lymph_node_location(location):
        if pd.isna(location):
            return "None"
        else:
            for key, synonyms in valid_locations.items():
                if any(synonym in location for synonym in synonyms):
                    return key
            return "无"

    # 假设你的DataFrame是df，包含淋巴结位置的列名是“淋巴结”
    df['淋巴结位置'] = df['淋巴结'].apply(process_lymph_node_location)
    # 删除原始的“淋巴结”列并赋值给df
    df = df.drop(columns=['淋巴结'])


    #df["罗音"].unique()----和呼吸音一种，删除该列
    df = df.drop(columns=["罗音"])
    # 删除一些列 先前忘记的
    df = df.drop(columns=["评价结果"])
    df = df.drop(columns=["老年人认知功能"])
    df = df.drop(columns=["齿列"])

    #df['甲胎蛋白'].unique()
    df["甲胎蛋白"] = df["甲胎蛋白"].replace("小于2", 1.8)

    df.to_csv("1.special_char_converted.csv", encoding="utf-8", index=False)
    return df
df1=special_Char(pd.read_csv("0.label_added.csv"))

# 4. Misiing data Clean in large scale

In [ ]:
def larger_missing_drop(filepath,outputpath):
    df=pd.read_csv(filepath, encoding="utf-8")
    print("大批量删除缺失值前有标记样本量：",df.shape[0]-df['MMSESCORE'].isnull().sum())
    print('删除缺失值之前',df.shape)

    
    #重要特征包含在0.6缺失值以上

    threshold = 0.6  # 设置缺失值阈值
    missing_ratio = df.isnull().mean()
    columns_to_drop = missing_ratio[missing_ratio > threshold]
    columns_to_drop1 = columns_to_drop.index.tolist()
    if 'MMSESCORE' in columns_to_drop1:
        columns_to_drop1.remove('MMSESCORE')
    print(columns_to_drop1)

    df=df.drop(columns=columns_to_drop1,inplace=False)
    df = df[df['文化程度'] != '不详']
    df= df[df['婚姻状况'] != '未说明的婚姻状况']
    df= df[df['老年健康状况自我评估'] != '说不清楚']
    print("大批量删除缺失值后有标记样本量：",df.shape[0]-df['MMSESCORE'].isnull().sum())
    
    
    # 检查删除列后，缺失值的分布  低于0.2
    threshold = 0.2  # 设置缺失值阈值
    missing_ratio = df.isnull().mean()
    columns_to_drop = missing_ratio[missing_ratio > threshold]
    columns_to_drop2 = columns_to_drop.index.tolist()
    print('删除后超过阈值的列',columns_to_drop2)

    df.to_csv(outputpath,encoding="gbk")
    return df
df2=larger_missing_drop("1.special_char_converted.csv","2.missing_cleaned_0.6.csv")

In [ ]:
df = pd.read_csv("2.missing_cleaned_0.6.csv", encoding="gbk")
unique_counts = df.nunique()
'''
分类值
'''
cate_features = unique_counts[unique_counts < 13].index.tolist()
print(cate_features)

value_freq_dict = {}
for col in cate_features:
    value_counts = df[col].value_counts()
    value_freq_dict[col] = {'value': value_counts.index.tolist(), 'frequency': value_counts.values.tolist()}
    missing_values = df[col].isnull().sum()
    value_freq_dict[col]['missing_values'] = missing_values
summary_df = pd.DataFrame(value_freq_dict)
summary_df.T.to_csv("3.description_cate_freq.csv", encoding="gbk", index=True)

'''
连续值
'''
continuous_cols= unique_counts[unique_counts >= 13].index.tolist()
description = df[continuous_cols].describe()
missing_ratio = df[continuous_cols].isnull().mean()
missing= df[continuous_cols].isnull().sum()
description['missing_ratio'] = missing_ratio
description['missing'] = missing
description.T.to_csv("3.description_continous.csv", encoding="gbk", index=True)

# 5. Misiing Data Fullfilling and convertment

In [ ]:
labeled_data=pd.read_csv("1.labeled_special_char_converted.csv")
labeled_data["文化程度"].value_counts()  #删除取值“不详”的列
labeled_data["婚姻状况"].value_counts()  #未说明的婚姻状况
labeled_data["老年健康状况自我评估"].value_counts() #删除  说不清楚

# 5.1 Cate columns


In [ ]:
# 分类的缺失值填充

cate=pd.read_csv("3.description_cate_freq.csv",encoding='gbk')
indexes = cate[cate['missing_values'] > 0]
cate_miss_all=indexes.iloc[:,0].tolist()
cate_for_chage=['缺齿左上','缺齿左下','缺齿右上','缺齿右下','龋齿左上','龋齿左下','龋齿右上', '龋齿右下','义齿左上','义齿左下','义齿右上','义齿右下',]
cate_for_ffill=list(set(cate_miss_all) - set(cate_for_chage))

df=pd.read_csv("2.missing_cleaned_0.6.csv",encoding="gbk")
df=df[df.nunique()[unique_counts < 13].index.tolist()]
df[cate_for_chage]=df[cate_for_chage].fillna(0)
df[cate_for_ffill]=df[cate_for_ffill].fillna(method='ffill')
df[cate_for_chage].value_counts()
df['淋巴结位置'] = df['淋巴结位置'].replace('None','无')
print(df.shape)
df.to_csv("4.filled_cate.csv",index=False)


unique_counts = df.nunique()
'''
分类值描述
'''
cate_features = unique_counts[unique_counts < 13].index.tolist()
value_freq_dict = {}
for col in cate_features:
    value_counts = df[col].value_counts()
    value_freq_dict[col] = {'value': value_counts.index.tolist(), 'frequency': value_counts.values.tolist()}
    missing_values = df[col].isnull().sum()
    value_freq_dict[col]['missing_values'] = missing_values
summary_df = pd.DataFrame(value_freq_dict)
summary_df.T.to_csv("4.filled_description_cate_freq.csv", encoding="gbk", index=True)



In [ ]:
df=pd.read_csv("4.filled_cate.csv")
print(df.shape)


data=pd.DataFrame()
#data["ID"]=df["ID"]
#data["MMSE"]=df["MMSESCORE"]

def single_col_map(colname,maps):
    data[colname] = df[colname].map(maps)
    return data

data=single_col_map("性别",{'男': 0, '女': 1, '未说明': 0})
data=single_col_map('民族',{'汉族': 1, '少数民族': 0})
data=single_col_map('文化程度', {'文盲或半文盲': 0,'小学': 1,'初中': 2,'技工学校': 3,'中等专业学校': 4,'高中': 5,'大学专科和专科学校': 6,'大学本科': 7,'研究生': 8})
data["AD8总分"]=0
mapping_dict = {'不是,没有变化': 0, '不知道,判断不出来': 0, '是,有变化': 1}
for col in ['问题1','问题2','问题3','问题4','问题5','问题6','问题7','问题8']:
    data=single_col_map(col,mapping_dict)
    data["AD8总分"]=data["AD8总分"]+data[col]
data=single_col_map('老年健康状况自我评估',{'满意': 3, '基本满意': 2, '不太满意': 1, '不满意': 0})
data=single_col_map('生活自理能力自我评估',{'可自理(0～3分)': 3, '轻度依赖(4～8分)': 2, '中度依赖(9～18分)': 1, '不能自理(≥19分)': 0})
data=single_col_map('老年人情感状态',{'粗筛阳性': 1, '粗筛阴性': 0})
data=single_col_map('锻炼频率',{'每天': 3, '每周一次以上': 2, '偶尔': 1, '不锻炼': 0})
data=single_col_map("饮酒频率",{'从不': 0, '偶尔': 1, '每天': 2, '经常': 3})
data=single_col_map('职业病危害因素接触史',{'无': 0, '有': 1})
data=single_col_map('听力',{'听见': 1, '听不清或无法听见': 0})
data=single_col_map('运动功能',{'可顺利完成':0, '无法独立完成任何一个动作':1})
data=single_col_map('桶状胸',{'是': 1, '否': 0})
data=single_col_map('心律',{'齐':0, '不齐':1, '绝对不齐':1})
data=single_col_map('腹部压痛',{'无':0, '有':1})
data=single_col_map('杂音',{'无':0, '有':1})
data=single_col_map('腹部包块',{'无':0, '有':1})
data=single_col_map('腹部肝大',{'无':0, '有':1})
data=single_col_map('腹部脾大',{'无':0, '有':1})
data=single_col_map('腹部移动性浊音',{'无':0, '有':1})
data=single_col_map('腹部B超是否异常',{'异常':1, '正常':0})
data=single_col_map('双肾B超是否异常',{'异常':1, '正常':0})


def multi_col_map(colname,special):
    dummies = pd.get_dummies(df[colname], prefix=colname)
    print(dummies.columns)
    if colname+'_'+str(special) in dummies.columns:
        dummies.drop(columns=[colname+'_'+str(special)], inplace=True)
    else:
        pass
    data[dummies.columns] =dummies
    return data

def multi_col_map1(colname):
    dummies = pd.get_dummies(df[colname], prefix=colname)
    print(dummies.columns)
    data[dummies.columns] =dummies
    return data
data=multi_col_map('血型','不详')
data=multi_col_map('职业','不便分类的其他从业人员')
data=multi_col_map('婚姻状况','None')
'''
data=multi_col_map('口唇','None')
data=multi_col_map('咽部','None')
data=multi_col_map('眼底异常位置','None')
data=multi_col_map('下肢水肿','None')
data=multi_col_map('巩膜位置','None')
data=multi_col_map('巩膜症状','None')
data=multi_col_map('呼吸音位置','None')
data=multi_col_map('呼吸音位置','None')
data=multi_col_map('呼吸音症状','None')
data=multi_col_map('呼吸音症状','None')
data=multi_col_map('淋巴结位置','None')
'''

data=multi_col_map1('口唇')
data=multi_col_map1('咽部')
data=multi_col_map1('眼底异常位置')
data=multi_col_map1('下肢水肿')
data=multi_col_map1('巩膜位置')
data=multi_col_map1('巩膜症状')
data=multi_col_map1('呼吸音位置')
data=multi_col_map1('呼吸音位置')
data=multi_col_map1('呼吸音症状')
data=multi_col_map1('呼吸音症状')
data=multi_col_map1('淋巴结位置')

cols=[ '嗜糖', '嗜油', '嗜盐', '荤食为主', '素食为主', '戒烟行为','缺齿左上', '缺齿左下', '缺齿右上', '缺齿右下', '龋齿左上', '龋齿左下', '龋齿右上', '龋齿右下', '义齿左上', '义齿左下', '义齿右上', '义齿右下', '左眼矫正视力', '右眼矫正视力', '尿蛋白', '尿糖', '尿酮体', '尿潜血', '心电图心房颤动', '心电图左心室电压', '心电图房室传导阻滞', '心电图T波改变', '心电图心电轴', '心电图ST-T改变', '心电图心率', '心电图心律类型', '胸脊柱侧弯畸形', '胸胸椎侧弯', '胸老年性肺间质改变', '胸冠脉支架植入术后', '胸气管受压', '胸胸主动脉粥样硬化', '胸肺炎症', '胸肺结节', '胸肺纹理增多', '胸心影增大', '胸主动脉硬化', '腹部前列腺钙化斑形成', '腹部胆充满型结石', '腹部胆附壁结石', '腹部胆囊多发性息肉', '腹部胆囊切除术后', '腹部胆总管扩张', '腹部胆结石', '腹部胆总管增宽', '腹部血管瘤', '腹部肝内钙化灶', '腹部内沉积物', '腹部胆囊泥沙样结石', '腹部息肉', '腹部胆囊壁隆起', '腹部囊肿', '腹部脂肪肝', '双肾肾柱', '双肾结石', '双肾囊肿', '双肾高回声', '双肾错构瘤']
data[cols]=df[cols]
#data.iloc[:5,-10:].head(20)
data.to_csv("5.transed_cate.csv",encoding='gbk',index=False)

# 5.2 Continous columns

(1) Error Data

In [ ]:
columns_to_check= [ '体温', '脉率', '呼吸频率', '舒张压R', '收缩压R', '舒张压L', '收缩压L', '身高', '体重', '腰围', '体质指数', '每次锻炼时间', '坚持锻炼时间', '吸烟年龄', '吸烟总支数', '左眼视力', '右眼视力',   '心率', '血红蛋白', '白细胞', '血小板', '空腹血糖', '尿微量白蛋白', '尿肌酐', '糖化血红蛋白', '癌胚抗原', '甲胎蛋白', '血清谷丙转氨酶', '血清谷草转氨酶', '总胆红素', '血清肌酐', '血尿素', '血尿酸', '总胆固醇', '甘油三酯', '低密度脂蛋白胆固醇', '高密度脂蛋白胆固醇',  'ID', '年龄']
data=pd.concat([pd.read_csv("2.missing_cleaned_0.6.csv",encoding='gbk')[columns_to_check+["MMSESCORE"]],pd.read_csv("5.transed_cate.csv",encoding='gbk')],axis=1)
data.to_csv("5.total_concated.csv",encoding='gbk',index=False)

# 异常值
data=pd.read_csv("5.total_concated.csv",encoding='gbk')

# 体温异常值
import re
temperature_indexes = data[data['体温'] < 30].index
temperature_values = data.loc[temperature_indexes, '体温']
temperatures=temperature_values.values.tolist()
#print(temperatures)
def extract_digits_before_and_after_decimal_point(temperature):
    pattern = r'(\d+)\.(\d)'
    match = re.match(pattern, temperature)
    if match:
        before_decimal = match.group(1)[-1]  
        after_decimal = match.group(2) 
        return float('3'+str(before_decimal)+'.'+str(after_decimal))
    else:
        return None  # 如果匹配失败，则返回None
tems=[]
for x in temperatures:
    #print(x)
    d=extract_digits_before_and_after_decimal_point(str(x))
    if d is not None:
        tems.append(d)
    else:
        tems.append(30)
print('*'*20,tems)
data.loc[temperature_indexes, '体温']=tems


height_indexes = data[data['身高'] < 50].index
height_values = data.loc[height_indexes, '身高']
height=height_values.values.tolist()
print(height)
data.loc[height_indexes, '身高']=[120.0, 136.2, 136.4, 140, 129.27, 105, 170, 170, 148.5, 180, 140, 170, 120.0, 180, 153, 119.0, 170, 170.0, 136.6]
data.loc[data['吸烟年龄'] < 0, '吸烟年龄'] = 0
data.loc[data['吸烟总支数'] < 0, '吸烟年龄'] = 0


xuetang= data[data['空腹血糖']>100].index
xuetang_values = data.loc[xuetang, '空腹血糖'].values.tolist()
print(xuetang_values)
data.loc[xuetang, '空腹血糖']=[2.520, 6.970, 5.36, 1.448, 1.24, 3.2, 6.35, 2.16, 5.017, 5.86]
print(data[data['空腹血糖']>100].index)

colname='甲胎蛋白'
indexs=data[data[colname]>10000].index
values = data.loc[indexs, colname].values.tolist()
print(values)
data.loc[indexs, colname]=[154, 137, 121, 121]


colname="腰围"
indexs=data[data[colname]>200].index
values = data.loc[indexs, colname].values.tolist()
print(values)
data.loc[indexs, colname]=[77.6, 70.73, 97.8, 96.98, 95.86, 99.6, 83, 77.6, 95.93, 98.5, 10.06, 58.5, 72.3, 94.4, 90.65, 85.00]


(2) Fullfilling 

In [ ]:
#data[columns_to_check].isnull().sum()
columns_to_check= [ '体温', '脉率', '呼吸频率', '舒张压R', '收缩压R', '舒张压L', '收缩压L', '身高', '体重', '腰围', '体质指数', '每次锻炼时间', '坚持锻炼时间', '吸烟年龄', '吸烟总支数', '左眼视力', '右眼视力',   '心率', '血红蛋白', '白细胞', '血小板', '空腹血糖', '尿微量白蛋白', '尿肌酐', '糖化血红蛋白', '癌胚抗原', '甲胎蛋白', '血清谷丙转氨酶', '血清谷草转氨酶', '总胆红素', '血清肌酐', '血尿素', '血尿酸', '总胆固醇', '甘油三酯', '低密度脂蛋白胆固醇', '高密度脂蛋白胆固醇',  'ID', '年龄']
data[columns_to_check].isnull().mean()
mode_values = data[columns_to_check].mode().iloc[0]
data[columns_to_check]= data[columns_to_check].fillna(mode_values)
data[columns_to_check]=(data[columns_to_check]-data[columns_to_check].min())/(data[columns_to_check].max()-data[columns_to_check].min())
data.to_csv("6.dataset_preprocessed_minmax.csv",encoding='gbk',index=False)

(3) Label convert 

In [ ]:
import pandas as pd
data = pd.read_csv("6.dataset_preprocessed_minmax.csv", encoding='gbk')
data['MMSESCORE'] = data['MMSESCORE'].map(lambda x: 1 if pd.notnull(x) and x >= 26 else (0 if pd.notnull(x) and x <= 26 else x))
data.to_csv("9.dataset26criteria.csv",encoding='gbk',index=False)